In [32]:
import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
import pctils as pc

import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
import pctils as pc
# SKLearn related imports
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
import spacy
import hashlib

from tqdm import tqdm
from collections import Counter
from spacy.matcher import Matcher
from sklearn.metrics import accuracy_score
from nltk.tokenize import WordPunctTokenizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin




In [29]:
# configuration
data_set = pc.getFile()

In [26]:
# configuration
label = "rates_count" # TODO
label2 = "helpful_count" # TODO

In [30]:
# Load the dataset
df = pd.read_csv(data_set) # never modify

# Show basic info
print(df.dtypes)            # all columns/types
print("Shape: ",df.shape)   # shape
df.head(10)



helpful_count    int64
rates_count      int64
dtype: object
Shape:  (2, 2)


,helpful_count,rates_count
0,4,4
1,2,4


In [49]:
# Create labels

# creates a new column helpful_count/rates_count
class TransformLabelsOi(TransformerMixin):
    def __init__(self, threshold = .75):
        self.helpful_count = 'helpful_count'
        self.rates_count = 'rates_count'
        self.threshold = threshold
        
    def isHelpful(self, helpful_count, rates_count):
        val = helpful_count/rates_count
        if val >= self.threshold:
            print("lolas")
            return 1
        return 0

    def transform(self, X, *_):
        _X = X.copy()

        out = _X[[self.helpful_count, self.rates_count]]

        ok = []
        for h, r in zip(_X[self.helpful_count], _X[self.rates_count]):
            ok += [self.isHelpful(h,r)]
        print(ok)    
        out['label'] = 0
        return out
    
    def fit(self, *_):
        return self


In [50]:
TransformLabelsOi().fit_transform(df)

lolas


TypeError: 'int' object is not iterable

In [28]:
# Show label distribution
pc.showColumnDistribution(df,label2)


Number of entries:  49992

Distribution of the column:


0       25237
1        8585
2        4085
3        2648
4        1682
        ...  
164         1
196         1
370         1
228         1
2878        1
Name: helpful_count, Length: 214, dtype: int64


Nan Values:  0


In [ ]:
# Clean up labels



In [ ]:

# Make sure text column is string. Also helps getting rid of nans
df[textColumn] = df[textColumn].astype(str)

# Get the text
docs = df[textColumn]
labels = df[labelColumn]

In [38]:
display(df.dtypes)

verifiedby        object
country           object
class             object
title             object
published_date    object
country1          object
country2          object
country3          object
country4          object
article_source    object
ref_source        object
source_title      object
content_text      object
category          object
lang              object
dtype: object